### Assignment 2

#### U95310908 Srikar Pusuluri

In [1]:
#importing libraries

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

np.random.seed(1)

In [2]:
import warnings 
warnings.filterwarnings("ignore")

In [3]:
X_train = pd.read_csv('phoneme_Xtrain.csv')
y_train = pd.read_csv('phoneme_ytrain.csv')
X_test = pd.read_csv('phoneme_Xtest.csv')
y_test = pd.read_csv('phoneme_ytest.csv')

In [4]:
X_train.head(10)

,0,1,2,3,4
0,-0.095847,2.016646,0.215286,-0.935430,0.245228
1,-0.604492,-0.299550,1.793911,-1.370485,0.546791
2,1.262095,-0.895578,-0.635078,-0.302393,-0.389642
3,0.474629,0.011931,1.760115,-1.129628,-0.460183
4,-0.179844,1.996739,-1.317550,-0.169985,-0.343790
5,0.586624,-0.777309,-0.669965,-0.309959,-0.409040
6,0.371966,1.049417,-1.257589,-0.163679,-0.454892
7,0.311302,-0.258565,-0.563124,-0.238080,-0.278539
8,-0.714154,-0.547797,0.537988,2.220928,-1.252006
9,-0.214842,2.117350,-1.556306,0.123835,-0.456656


### Modelling the Data

In [5]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})
performance


,model,Accuracy,Precision,Recall,F1


- Our primary goal is to identify the correct sound i.e, Nasal or Oral., with a high degree of accuracy, then accuracy is the best scoring metric. Therefore we are considering accuracy as the scoring metric.

## Fitting a Logistic Regression Model

In [6]:
log_reg_model = LogisticRegression()
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [7]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Default Logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Logistic Regression using RandomSearchCV 

In [8]:
hyperparam= {'C':[0.001, 0.01, 0.1, 1, 10, 100],
                      'penalty':['l2', 'l1']
                     }

score_measure = "accuracy"
#create a logistic regression model
logistic_model = LogisticRegression()

#create a random search cv object
random_search = RandomizedSearchCV(estimator=logistic_model,
                                     param_distributions=hyperparam,
                                     cv=5,scoring=score_measure,
                                     n_iter=10,
                                     n_jobs=-1,
                                   )

_ = random_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {random_search.best_score_}")
print(f"... with parameters: {random_search.best_params_}")

bestRecallTree = random_search.best_estimator_


The best accuracy score is 0.7506746288258893
... with parameters: {'penalty': 'l2', 'C': 100}


In [10]:
model_predict = random_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_predict)
logreg_acc_rand= accuracy_score(y_test, model_predict)

In [11]:
c_matrix = confusion_matrix(y_test, random_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"LogReg - RandomCV", 
                                                    'Accuracy': logreg_acc_rand, 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Logistic Regression using GridsearchCV

In [12]:
score_measure = "accuracy"

Log_Reg = LogisticRegression()

#Define the parameter grid
param_grid = {
    'penalty':['l1','l2'],
    'C':[0.1,1,10]
}

#Create the grid search object
grid_searchCV = GridSearchCV(estimator=Log_Reg, param_grid=param_grid, cv = 5, scoring= score_measure )


_ = grid_searchCV.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_searchCV.best_score_}")
print(f"... with parameters: {grid_searchCV.best_params_}")

bestRecallTree = grid_searchCV.best_estimator_

The best accuracy score is 0.7506746288258893
... with parameters: {'C': 1, 'penalty': 'l2'}


In [13]:
model_pred = grid_searchCV.predict(X_test)
c_matrix = confusion_matrix(y_test, model_pred)
logreg_acc_grid= accuracy_score(y_test, model_pred)

In [15]:
c_matrix = confusion_matrix(y_test, grid_searchCV.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"LogReg - GridCV", 
                                                    'Accuracy': logreg_acc_grid, 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Decision Tree classification model using defaults 

In [16]:
decitree = DecisionTreeClassifier().fit(X_train, np.ravel(y_train))

In [17]:
c_matrix = confusion_matrix(y_test, decitree.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree using Defaults", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### Decision Tree using RandomsearchCV

In [18]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,70),  
    'min_samples_leaf': np.arange(1,70),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.8286350437610942
... with parameters: {'min_samples_split': 2, 'min_samples_leaf': 3, 'min_impurity_decrease': 0.0026, 'max_leaf_nodes': 102, 'max_depth': 12, 'criterion': 'entropy'}


In [19]:
model = rand_search.predict(X_test)
dtree_acc_rand= accuracy_score(y_test, model)

In [20]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree- RandCV", 
                                                    'Accuracy': (dtree_acc_rand), 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### Decision Tree using GridsearchCV

In [21]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(30,36),  
    'min_samples_leaf': np.arange(6,12),
    'min_impurity_decrease': np.arange(0.0048, 0.0054, 0.0001),
    'max_leaf_nodes': np.arange(162,168), 
    'max_depth': np.arange(15,21), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 9072 candidates, totalling 45360 fits
The best accuracy score is 0.8068868573070255
... with parameters: {'criterion': 'entropy', 'max_depth': 15, 'max_leaf_nodes': 162, 'min_impurity_decrease': 0.0048, 'min_samples_leaf': 6, 'min_samples_split': 30}


In [22]:
model = rand_search.predict(X_test)
decitree_acc_grid= accuracy_score(y_test, model)

In [23]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]


performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree- GridCV", 
                                                    'Accuracy': (decitree_acc_grid), 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM model using linear kernal

In [24]:
from sklearn.svm import SVC
svm_linear_model = SVC(kernel="linear")
_ = svm_linear_model.fit(X_train, np.ravel(y_train))

In [25]:
model_preds = svm_linear_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM Linear Model - RandomSearch CV

In [26]:
score_measure = "accuracy"
# Create parameter grid
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}

#create a SVM classifier
classifier = SVC(kernel='linear')

# Create the random search model
rand_search = RandomizedSearchCV(classifier, param_grid, cv=5, n_iter=50, scoring=score_measure, n_jobs=-1, verbose=1)


_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best accuracy score is 0.751584736458686
... with parameters: {'max_iter': 1500, 'C': 1}


In [27]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM Linear- RandCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM Linear using GridSearchCV

In [28]:
score_measure = "accuracy"
kfolds = 5
#create a SVM classifier
clf = SVC(kernel='linear')
#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear']}  
#instantiate the GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring= score_measure)


_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_


The best accuracy score is 0.7479566185448537
... with parameters: {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}


In [29]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM Linear- GridCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM classification model using rbf kernal

In [30]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale')
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [31]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM rbf", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM rbf - RandomSearchCV

In [32]:
score_measure = "accuracy"
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}

#create a SVM classifier
svm_clf = SVC(kernel='rbf')

# Create the random search model
random_search_rbf = RandomizedSearchCV(svm_clf, param_grid, cv=5, n_iter=50, scoring=score_measure, n_jobs=-1, verbose=1)


_ = random_search_rbf.fit(X_train, y_train)

print(f"The best {score_measure} score is {random_search_rbf.best_score_}")
print(f"... with parameters: {random_search_rbf.best_params_}")

bestRecallTree = random_search_rbf.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best accuracy score is 0.8381650095935811
... with parameters: {'max_iter': 1500, 'C': 10}


In [33]:
c_matrix = confusion_matrix(y_test, random_search_rbf.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM rbf- RandCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### SVM rbf - GridSearchCV

In [34]:
score_measure = "accuracy"
clf = SVC(kernel='rbf')

#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['rbf']} 


grid_CV = GridSearchCV(clf, param_grid, cv=5, scoring= score_measure)


_ = grid_CV.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_CV.best_score_}")
print(f"... with parameters: {grid_CV.best_params_}")

bestRecallTree = grid_CV.best_estimator_


The best accuracy score is 0.8612819486769068
... with parameters: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}


In [35]:
c_matrix = confusion_matrix(y_test, grid_CV.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM rbf- GridCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM classification model using polynomial kernal

In [36]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10, probability = True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [37]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"poly svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### SVM Poly - RandomSearchCV

In [38]:
score_measure = "accuracy"
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}

svm_clf = SVC(kernel='poly')

# Create the random search model
rand_search_polym = RandomizedSearchCV(svm_clf, param_grid, cv=5, n_iter=50, scoring=score_measure, n_jobs=-1, verbose=1)


_ = rand_search_polym.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_polym.best_score_}")
print(f"... with parameters: {rand_search_polym.best_params_}")

bestRecallTree = rand_search_polym.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best accuracy score is 0.7615497481043699
... with parameters: {'max_iter': 2000, 'C': 1}


In [39]:
c_matrix = confusion_matrix(y_test, rand_search_polym.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM Poly- RandCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM Poly - GridSearchCV

In [40]:
score_measure = "accuracy"
clf = SVC(kernel='poly')

#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['poly']}  
#instantiate the GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring= score_measure)


_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

The best accuracy score is 0.7620083931008301
... with parameters: {'C': 100, 'gamma': 1, 'kernel': 'poly'}


In [43]:
model = grid_search.predict(X_test)
grid_acc_poly= accuracy_score(y_test, model)

In [44]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM Poly- GridCV", 
                                                    'Accuracy': [grid_acc_poly], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Neural Network with Default Parameters

In [45]:
from sklearn.neural_network import MLPClassifier

In [46]:
%%time
ann = MLPClassifier(hidden_layer_sizes=(60,50,40), solver='adam', max_iter=200)
_ = ann.fit(X_train, y_train)

CPU times: total: 16.3 s
Wall time: 2.43 s


In [47]:
%%time
y_pred = ann.predict(X_test)

CPU times: total: 0 ns
Wall time: 6.54 ms


In [48]:
nn_acc= accuracy_score(y_test, y_pred)
print(nn_acc)

0.8372379778051788


In [49]:
performance = pd.concat([performance, pd.DataFrame({'model':"Neural Network", 
                                                    'Accuracy': nn_acc, 
                                                    'Precision': precision_score(y_test, y_pred), 
                                                    'Recall': recall_score(y_test, y_pred), 
                                                    'F1': f1_score(y_test, y_pred)
                                                     }, index=[0])])

### Neural Network - RandomsearchCV

In [50]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'solver': 'adam', 'max_iter': 5000, 'learning_rate_init': 0.001, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (70, 50, 40), 'alpha': 0, 'activation': 'tanh'}
CPU times: total: 33.1 s
Wall time: 2min 7s


In [51]:
%%time
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.77      0.85      1139
           1       0.62      0.88      0.73       483

    accuracy                           0.80      1622
   macro avg       0.78      0.83      0.79      1622
weighted avg       0.84      0.80      0.81      1622

CPU times: total: 0 ns
Wall time: 11.4 ms


In [52]:
nn_acc_rand= accuracy_score(y_test, y_pred)
print(nn_acc_rand)

0.8033292231812577


In [53]:
performance = pd.concat([performance, pd.DataFrame({'model':"Neural Network RandomCV", 
                                                    'Accuracy': nn_acc_rand, 
                                                    'Precision': precision_score(y_test, y_pred), 
                                                    'Recall': recall_score(y_test, y_pred), 
                                                    'F1': f1_score(y_test, y_pred)
                                                     }, index=[0])])

### Neural Networks With GridSearchCV

In [54]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (30,), (50,), (70,), (90,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [.5, .7, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [0.005, 0.01, 0.15],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
{'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling', 'learning_rate_init': 0.01, 'max_iter': 5000, 'solver': 'adam'}
CPU times: total: 5.72 s
Wall time: 57.5 s


In [55]:
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.76      0.84      1139
           1       0.61      0.89      0.72       483

    accuracy                           0.80      1622
   macro avg       0.78      0.82      0.78      1622
weighted avg       0.84      0.80      0.81      1622



In [57]:
nn_grid= accuracy_score(y_test, y_pred)

In [58]:
performance = pd.concat([performance, pd.DataFrame({'model':"Neural Network GridCV", 
                                                    'Accuracy': nn_grid, 
                                                    'Precision': precision_score(y_test, y_pred), 
                                                    'Recall': recall_score(y_test, y_pred), 
                                                    'F1': f1_score(y_test, y_pred)
                                                     }, index=[0])])

#### Deep Neural Network using Keras

In [59]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [60]:
from __future__ import print_function
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt

from sklearn import datasets

In [61]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [62]:
%%time

# create model stucture
model = keras.models.Sequential()
model.add(keras.layers.Input(5))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax')) # final layer, 10 categories

# compile
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# if you want to overide the defaults for the optimizer....
#adam = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


CPU times: total: 109 ms
Wall time: 126 ms


In [63]:
%%time

# fit the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20, batch_size=100)

Epoch 1/20
23/23 [==============================] - 1s 17ms/step - loss: 1.9232 - accuracy: 0.4995 - val_loss: 1.2406 - val_accuracy: 0.7306
Epoch 2/20
23/23 [==============================] - 0s 6ms/step - loss: 0.9663 - accuracy: 0.6455 - val_loss: 0.6039 - val_accuracy: 0.7774
Epoch 3/20
23/23 [==============================] - 0s 7ms/step - loss: 0.5491 - accuracy: 0.7652 - val_loss: 0.4995 - val_accuracy: 0.7836
Epoch 4/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4936 - accuracy: 0.7724 - val_loss: 0.4944 - val_accuracy: 0.7620
Epoch 5/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4711 - accuracy: 0.7874 - val_loss: 0.4611 - val_accuracy: 0.7855
Epoch 6/20
23/23 [==============================] - 0s 7ms/step - loss: 0.4585 - accuracy: 0.7951 - val_loss: 0.4719 - val_accuracy: 0.7811
Epoch 7/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4462 - accuracy: 0.8055 - val_loss: 0.4278 - val_accuracy: 0.7996
Epoch 8/20
23/23 [=

In [64]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores
# In results, first is loss, second is accuracy

[0.39232808351516724, 0.8027126789093018]

In [65]:
grid= accuracy_score(y_test, y_pred)
print(grid)

0.7977805178791615


In [66]:
# let's format this into a better output...

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


performance = pd.concat([performance, pd.DataFrame({'model':"Deep Network using keras", 
                                                    'Accuracy': scores[1]}, index=[0])])

loss: 0.39
accuracy: 80.27%


#### Wide & Deep Network

In [67]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=5))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [68]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [69]:
# Fit the model

history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20, batch_size=100)

Epoch 1/20
23/23 [==============================] - 1s 17ms/step - loss: 0.6932 - accuracy: 0.7285 - val_loss: 0.4098 - val_accuracy: 0.7651
Epoch 2/20
23/23 [==============================] - 0s 6ms/step - loss: 0.5010 - accuracy: 0.7684 - val_loss: 0.3885 - val_accuracy: 0.8021
Epoch 3/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4557 - accuracy: 0.8033 - val_loss: 0.4327 - val_accuracy: 0.7861
Epoch 4/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4198 - accuracy: 0.8250 - val_loss: 0.4765 - val_accuracy: 0.7910
Epoch 5/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4049 - accuracy: 0.8223 - val_loss: 0.4921 - val_accuracy: 0.7466
Epoch 6/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4306 - accuracy: 0.8128 - val_loss: 0.3884 - val_accuracy: 0.7978
Epoch 7/20
23/23 [==============================] - 0s 5ms/step - loss: 0.3853 - accuracy: 0.8300 - val_loss: 0.3938 - val_accuracy: 0.7959
Epoch 8/20
23/23 [=

In [70]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.36638206243515015, 0.8125770688056946]

In [71]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


performance = pd.concat([performance, pd.DataFrame({'model':"Keras - Wide & Deep Network", 
                                                    'Accuracy': scores[1]}, index=[0])])

loss: 0.37
accuracy: 81.26%


#### RandomGridSearch

In [72]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=5)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotUniform(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(2, activation='softmax'))
    ann.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return ann


CPU times: total: 15.6 ms
Wall time: 46.7 ms


In [73]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=5,
    dropout = 0.0
)


In [74]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.0005, 0.001, 0.005],
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)],
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10, 50, 100],
    'optimizer':["adam",'sgd']
}
keras_clf.get_params().keys()



dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [75]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(3000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)

8/8 [==============================] - 0s 1ms/step


In [76]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.005,
 'optimizer': 'sgd',
 'model__hidden_layer_sizes': (100, 90),
 'model__dropout': 0,
 'epochs': 50,
 'batch_size': 100}

In [77]:
best_net = rnd_search_cv.best_estimator_
print(rnd_search_cv.best_params_)

{'optimizer__learning_rate': 0.005, 'optimizer': 'sgd', 'model__hidden_layer_sizes': (100, 90), 'model__dropout': 0, 'epochs': 50, 'batch_size': 100}


In [78]:
%%time
y_pred = best_net.predict(X_test)
print(classification_report(y_test, y_pred))

17/17 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.90      0.68      0.77      1139
           1       0.52      0.81      0.63       483

    accuracy                           0.72      1622
   macro avg       0.71      0.75      0.70      1622
weighted avg       0.78      0.72      0.73      1622

CPU times: total: 641 ms
Wall time: 146 ms


In [79]:
keras_rand_acc= accuracy_score(y_test, y_pred)
print(keras_rand_acc)

0.718249075215783


In [80]:

performance = pd.concat([performance, pd.DataFrame({'model':"RandomGrid using Keras", 
                                                    'Accuracy': keras_rand_acc}, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Default Logistic,0.717016,0.516000,0.801242,0.627737
0,LogReg - RandomCV,0.717016,0.516000,0.801242,0.627737
0,LogReg - GridCV,0.717016,0.516000,0.801242,0.627737
0,Decision Tree using Defaults,0.844636,0.694118,0.855072,0.766234
0,Decision Tree- RandCV,0.818126,0.663763,0.788820,0.720908
0,Decision Tree- GridCV,0.818126,0.627019,0.884058,0.733677
0,SVM Linear,0.709001,0.506765,0.853002,0.635802
0,SVM Linear- RandCV,0.688656,0.487028,0.855072,0.620586
0,SVM Linear- GridCV,0.709618,0.507371,0.855072,0.636854
0,SVM rbf,0.822441,0.641509,0.915114,0.754266


## Analysis

- The performance metrics and accuracy ratings for different machine learning models are shown in the above table for the Phoneme dataset. Comparisons have been made between the models of logistic regression, decision trees, support vector machines (SVM), and neural networks. 

- The highest accuracy score of 0.848952 was achieved by the SVM-rbf GridCV model, followed by the Decision model with Default parameters  with an accuracy of 0.844636. The Deep Network using Keras and the Keras Wide & Deep Network both achieved accuracy scores of 0.802713 and 0.812577 respectively. The Logistic Regression models all achieved the same accuracy score of 0.717016. The SVM randomCV model achieved the lowest accuracy score of 0.688656 followed by SVM Linear model with accuracy score of 0.709001.

- The Keras Models (Deep Neural Network, Wide & Deep Network) have the moderate accuracy scores are little higher compared to other models. Accuracy scores of neural network with default parameters is 0.837238 has the third highest accuracy score of all the models.